In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import datetime as dt
from bokeh.io import export_png

In [2]:
bpl.output_notebook()

Loading BokehJS ...

In [23]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/17/2020"

In [24]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

In [20]:
url = "http://187.191.75.115/gobmx/salud/datos_abiertos/historicos/datos_abiertos_covid19_"

In [21]:
fechas_zip = fechas[:-1].strftime("%d.%m.%Y")

In [22]:
for fecha in fechas_zip:
    r = requests.get(url + fecha + ".zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path="./datos_federales/")

In [8]:
url + fechas_zip[0] + ".zip"

'http://187.191.75.115/gobmx/salud/datos_abiertos/historicos/datos_abiertos_covid19_12.04.2020.zip'

Adquisición de los últimos datos, se necesita ejecutar diario para estar actualizado, si no se ejecuta diario la adquisición histórica puede ayudar a obtener los datos que faltan

In [9]:
urlnew = "http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"

In [10]:
r = requests.get(urlnew)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="./datos_federales/")

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [25]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [26]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [27]:
len(frames)

36

In [28]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [29]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [30]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

Agrupación de registros por id único

In [31]:
grupos_id = all_data.groupby("ID_REGISTRO")

Definición de dataframe por pacientes con fechas relevantes a su historia

In [32]:
pacientes = grupos_id[["FECHA_SINTOMAS","FECHA_ARCHIVO","FECHA_INGRESO","FECHA_DEF"]].min()

Definición de función para extracción de fecha de primer evento en la historia de un paciente

In [33]:
def fechador(frame,columna,valor):
    sframe = frame.sort_values("FECHA_ARCHIVO")
    fframe = sframe[sframe[columna]==valor]
    if(len(fframe)==0):
        return None
    ind_s = sframe.index[0]
    ind_f = fframe.index[0]
    if(ind_s==ind_f):
        return sframe["FECHA_INGRESO"].min()
    else:
        return fframe.loc[ind_f,"FECHA_ARCHIVO"]

Obtención de fechas de primer evento para todos los pacientes para algunos eventos importantes, esta parte del código tarda bastante y no está optimizada

In [ ]:
fec_pos = grupos_id.apply(lambda x: fechador(x,"RESULTADO",1))
fec_neg = grupos_id.apply(lambda x: fechador(x,"RESULTADO",2))
fec_hos = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",1))
fec_amb = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",2))
fec_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",1))
fec_noint = grupos_id.apply(lambda x: fechador(x,"INTUBADO",2))
fec_noa_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",97))
fec_ign_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",99))
fec_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",1))
fec_noneu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",2))
fec_ign_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",99))
fec_uci = grupos_id.apply(lambda x: fechador(x,"UCI",1))
fec_nouci = grupos_id.apply(lambda x: fechador(x,"UCI",2))
fec_noa_uci = grupos_id.apply(lambda x: fechador(x,"UCI",97))
fec_ign_uci = grupos_id.apply(lambda x: fechador(x,"UCI",99))

In [ ]:
pacientes["FECHA_POSITIVO"] = fec_pos
pacientes["FECHA_NEGATIVO"] = fec_neg
pacientes["FECHA_HOSPITALIZACION"] = fec_hos
pacientes["FECHA_AMBULATORIO"] = fec_amb
pacientes["FECHA_INTUBACION"] = fec_int
pacientes["FECHA_NO_INTUBACION"] = fec_noint
pacientes["FECHA_NO_APL_INTUBACION"] = fec_noa_int
pacientes["FECHA_SE_IGN_INTUBACION"] = fec_ign_int
pacientes["FECHA_NEUMONIA"] = fec_neu
pacientes["FECHA_NO_NEUMONIA"] = fec_noneu
pacientes["FECHA_SE_IGN_NEUMONIA"] = fec_ign_neu
pacientes["FECHA_UCI"] = fec_uci
pacientes["FECHA_NO_UCI"] = fec_nouci
pacientes["FECHA_NOAPL_UCI"] = fec_noa_uci
pacientes["FECHA_SE_IGN_UCI"] = fec_ign_uci

In [ ]:
pacientes.rename(columns={"FECHA_ARCHIVO":"FECHA_APARICION"},inplace=True)

Exportar las historias por paciente a un archivo

In [ ]:
fecha_file = fechas[-1].strftime("%d_%m_%Y")
pacientes.to_csv("./datos_procesados/historia_pacientes_" + fecha_file + ".csv")

Búsqueda de pacientes en específico con alguna condición

In [ ]:
columna = "FECHA_UCI"
pacientes[(pacientes[columna].notna())&(pacientes["FECHA_INGRESO"]!=pacientes[columna])]

Graficación de la historia de un paciente

In [ ]:
caso_tmp = all_data[all_data["ID_REGISTRO"]=="00b7d8"][["ID_REGISTRO","RESULTADO","TIPO_PACIENTE","FECHA_ARCHIVO","FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF","INTUBADO","NEUMONIA","UCI"]]
caso_tmp

In [ ]:
no_positivo, positivo, pendiente = "#00BFA5","#DD5246","#F8C600"
ambulatorio, hospitalizado, no_esp = "#FF9633", "#E10C0D", "#D8DBDD"
si, no, no_aplica, se_ignora = "#970A03", "#1F7249", "#E0E0E0", "#A6A6A6"

In [ ]:
registro = caso_tmp["ID_REGISTRO"].min()
inicio_sint = caso_tmp["FECHA_SINTOMAS"].min()
deceso = caso_tmp["FECHA_DEF"].min()
recuperacion = inicio_sint+pd.Timedelta('14 days')
final = fechas[-1]

In [ ]:
p = bpl.figure(x_axis_type="datetime",x_range=(inicio_sint-(final-inicio_sint)*0.1,final+(final-inicio_sint)*0.4),
               y_range=[" ","UCI","Intubado","Neumonía","Hospitalización","Resultado pruebas","Síntomas y defunción",""],
               plot_width=1000,plot_height=300,
               title="Paciente " + registro,
               toolbar_location=None)

In [ ]:
p.line(x=[inicio_sint]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="blue",line_width=3,legend_label="Inicio síntomas")
if not pd.isnull(deceso):
    p.line(x=[deceso]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],color="black",line_width=3,legend_label="Defunción")
    p.line(x=[inicio_sint,deceso],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
else:
    deceso = final
    p.line(x=[inicio_sint,final],y=[["Síntomas y defunción"],["Síntomas y defunción"]],color="black",line_width=0.8)
if (recuperacion <= final)&(recuperacion <= deceso):
    p.line(x=[inicio_sint+pd.Timedelta('14 days')]*2,y=[["Síntomas y defunción",-0.45],["Síntomas y defunción",0.45]],line_dash="dotted",color="purple",line_width=3,legend_label="14 días")

In [ ]:
for valor,color,label in zip([3,2,1],[pendiente,no_positivo,positivo],["Prueba Pendiente","Prueba No positiva","Prueba Positiva"]):
    data_temporal = caso_tmp[(caso_tmp["RESULTADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Resultado pruebas"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

In [ ]:
for valor,color,label in zip([1,2,99],[ambulatorio, hospitalizado,no_esp],["Ambulatorio","Hospitalizado","No especificado"]):
    data_temporal = caso_tmp[(caso_tmp["TIPO_PACIENTE"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Hospitalización"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

In [ ]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["INTUBADO"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Intubado"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

In [ ]:
for valor,color,label in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si],["No aplica","Se ignora","No especificado","No","Si"]):
    data_temporal = caso_tmp[(caso_tmp["NEUMONIA"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    if len(data_temporal)>0:
        p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["Neumonía"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white",legend_label=label)

In [ ]:
for valor,color in zip([97,98,99,2,1],[no_aplica, se_ignora, no_esp, no, si]):
    data_temporal = caso_tmp[(caso_tmp["UCI"]==valor)&(caso_tmp["FECHA_ARCHIVO"]<=final)]
    p.rect(x=data_temporal["FECHA_ARCHIVO"],y=["UCI"]*len(data_temporal),height=0.9,width=fechas[1]-fechas[0],color=color,line_color="white")

In [ ]:
bpl.show(p)

In [1096]:
export_png(p, filename="./graficas/" + registro + "_" + fecha_file + ".png")

'E:\\proyectos\\morlan\\covid_plots\\graficas\\167f53_12_05_2020.png'